In [41]:
# just importing a random selection of things; we can fix this later
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LinearRegression, LogisticRegressionCV
%matplotlib inline

from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.grid_search import GridSearchCV

from pprint import pprint 

from sklearn.ensemble import BaggingClassifier
from sklearn import model_selection
from random import seed
from sklearn.model_selection import cross_val_predict

from sklearn.ensemble import RandomForestClassifier
from itertools import product
from collections import OrderedDict
from itertools import product

from sklearn import model_selection
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import train_test_split
from __future__ import print_function # For Python 2 / 3 compatability
from sklearn.metrics import confusion_matrix
import itertools

import json
from copy import copy

import warnings
from sklearn import preprocessing
%autosave 300
warnings.filterwarnings("ignore")

Autosaving every 300 seconds


In [42]:
#Split function used to split train and test
def split (data, percentage):
    np.random.seed(9001) 
    msk = np.random.rand(len(data)) < percentage
    train = data[msk]
    test = data[~msk]
    return train, test

# Data Cleaning and Preliminary Exploration

## Preliminary Exploration

In [43]:
# load ADNIMERGE.csv, the main dataset (we will add our own variables to it). 
ADNI_merge = pd.read_csv('ADNIMERGE.csv')
ADNI_merge.head()

# select only the rows that correspond to ADN2
ADNI2= ADNI_merge[ADNI_merge['COLPROT']=='ADNI2']
ADNI2 = ADNI2[ADNI2['VISCODE']== 'bl']
print(ADNI2.shape)

(789, 94)


In [44]:
switch = 3
# switch = 0  #ADNI2
# switch = 1  #ADNI2+DRUG
# switch = 2  #ADNI2+PROTEIN
# switch = 3  #ADNI2+PROTEIN_DRUG

In [45]:
# MERGING DRUG
if switch == 1:

        # checking out the number of records for patients in ADNI2 phase, compared with other phases
        ADMCPATIENTDRUGCLASSES_20170512 = pd.read_csv('ADMCPATIENTDRUGCLASSES_20170512.csv')
        print(len(ADMCPATIENTDRUGCLASSES_20170512['Phase']))
        print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI2']))
        print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI3']))
        print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNIGO']))

        # select only ADNI2 data within MEDHIST
        drug_adni2 = ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI2']

        # renaming VISCODE2 column to VISCODE; this is okay, since we're only using ADNI2 data. 
        drug_adni2 = drug_adni2.rename(columns={'VISCODE2': 'VISCODE'})

        # create a list of all the drugs in ADNI2 
        drug_list_allviscode = [x for x in drug_adni2['NA']]
        # use a set to find all the unique drug names in the dataset
        all_drug_list_all_viscode = set(drug_list_allviscode)

        # create a list of blood-thinners from a google search
        blood_thinners_original = ['DABIGATRAN','PRADAXA','RIVAROXABAN','XARELTO','APIXABAN','ELIQUIS','HEPARIN',\
                                    'WARFARIN', 'COUMADIN', 'ASPIRIN']

        # a more extensive list of blood thinners found online
        blood_thinners = ['ACTRON DAMASON P LIQUPRIN PRESALIN ADVIL DARVON LODINE PROFEN AGGRENOX DASIN LORTAB RELAFEN \
        ALEVE DAYPRO MAGNAPRIN ROBAXISAL ALKA-SELTZER DHC PLUS MARNAL ROXIPRIN ANACIN DIAGESIC MEASURIN RUFEN ANAPROX\
        DICLOFENAC MECLOFENAMATE SALSALATE ANEXSIA DIPYRIDAMOLE MEFENAMIC SALETO ANODYNOS DISALCID MELOXICAM SALOCOL\
        ANSAID DOLABID MEPROBAMATE SOMA COMPOUND A.P.C. DOLPRIN MIDOL SPRIX NASAL SPRAY ARTROTEC DOXAPHENE MOBIC ST.\
        JOSEPH A.S.A. DRISTAN MOMENTUM SULINDAC ASCRIPTIN EASPRIN MOTRIN SUPAC ASPERGUM ECOTRIN NABUMETONE SYNALGOS \
        ASPIRIN EMAGRIN FORTE NAPRELAN TALWIN AXOTAL EMPIRIN NAPRAPAC TICAGRELOR BAC EQUAGESIC NAPROSYN TICLOPIDINE BAYER\
        EQUAZINE NAPROXEN TICLID BEXOPHENE ETODOLAC NORGESIC TOLECTIN BEXTRA EXCEDRIN NUPRIN TOLMETIN BRILINTA FELDENE\
        ORUDIS TORADOL BUFFERED ASPIRIN FENOPROFEN ORUVAIL TRENTAL BUFFAPRIN FIOGESIC OXAPROZIN TRIGESIC BUFFERIN FIORGEN\
        PF PABALATE TRILISATE BUFFINOL FIORINAL  ULTRAPRIN CAMA ARTHRITIS PERCODAN UNIPRO CATAFLAM GINKO BILOBA PERSANTINE\
        VANGUISH CELEBREX GEMNISYN PERSISTIN VICOPROFEN CHERACOL CAPSULES IBUPROFEN PIROXICAM VIMOVO CLINORIL INDOCIN PLAVIX\
        CLOPIDOGREL INDOMETHACIN PLETAL VOLTAREN COMBUNOX KETOPROFEN PONSTEL ZIPSOR CONGESPIRIN KETOROLAC PREVACID NAPRAPAC\
        ZORPIN COUMADIN WARFARIN PRADAXA DABIGATRAN ETEXILATE'.split(' ')]

        # combine the 2 blood thinner lists to create a main list of all blood thinners
        blood_thinners = blood_thinners[0]
        blood_thinners = blood_thinners + blood_thinners_original

        # create a list of the supplements of interest, including all possible name variations
        supplements_original = ['CALCIUM','CA', 'CA2+','VITAMIN D','VIT D','VITA D']

        # split the NA column to identify all listed medications
        drug_adni2['DRUGS'] = drug_adni2["NA"].apply(lambda x:str(x).split(","))


        for sup in ['VITAMIN D', 'CALCIUM', 'VIT D']:
            drug_adni2[sup] = drug_adni2.apply(lambda x: np.sum(sup in x['DRUGS']), axis=1)

        # function to identify whether blood thinners are in the medical history dataset entries
        def ANY_BT(row):
            ANY=0
            for bt in blood_thinners:
                if bt in row["DRUGS"]:
                    ANY=1
            return ANY

        # any_bt column to identify if patient is taking a blood thinner (yes = 1)
        drug_adni2["ANY BT"] = drug_adni2.apply(ANY_BT, axis=1)
        # column to identify if patient is taking vitamin D supplement (yes = 1)
        drug_adni2["VITAMIN D merge"] = drug_adni2.apply(lambda x: np.sum(x["VITAMIN D"]==1 or x["VIT D"]==1), axis=1)
        drug_adni2 = drug_adni2.drop(["VITAMIN D", "VIT D"], axis=1)
        drug_adni2
        print(drug_adni2.shape)

        cholesterol_drugs = 'ATORVASTATIN, LIPITOR, FLUVASTATIN, LESCOL, LESCOL XL, LOVASTATIN, MEVACOR, ALTOPREV, PRAVASTATIN, PRAVACHOL, ROSUVASTATIN, CRESTOR, SIMVASTATIN, ZOCOR, PITAVASTATIN, LIVALO'
        cholesterol_drugs = cholesterol_drugs.split(', ')

        def ANY_CHOL(row):
               ANY=0
               for ct in cholesterol_drugs:
                   if ct in row["DRUGS"]:
                       ANY=1
               return ANY
    
        drug_adni2['ANY CHOL'] = drug_adni2.apply(ANY_CHOL, axis=1)
        
        # define a new dataframe with medications, VISCODE, RID
        drug_adni2_three_col = drug_adni2[['CALCIUM','ANY BT','VITAMIN D merge', 'ANY CHOL','RID', "VISCODE"]]
        
        print(ADNI2.shape)
        ADNI2= pd.merge(ADNI2,drug_adni2_three_col, on=['RID', 'VISCODE'])
        ADNI2.head()
        print(ADNI2.shape)

In [46]:
# MERGING PROTEIN
if switch == 2:
    ADNI_bio = pd.read_csv('UPENNBIOMK9_04_19_17.csv')
    ADNI2_bio = ADNI_bio[ADNI_bio['PHASE']=='ADNI2']
    ADNI2_bio_bl = ADNI2_bio[ADNI2_bio['VISCODE2']=='bl']
    ADNI2_bio_bl = ADNI2_bio_bl[['ABETA','TAU','PTAU', 'RID']]
    ADNI2_bio_bl.loc[ADNI2_bio_bl['ABETA']=='>1700', 'ABETA'] = 1700
    ADNI2_bio_bl.loc[ADNI2_bio_bl['PTAU']=='<8', 'PTAU'] = 8
    print("ADNI2 shape: ", ADNI2_bio_bl.shape)

    ADNI2 = ADNI2.merge(ADNI2_bio_bl, how='inner', left_on='RID', right_on='RID')
    print(ADNI2.shape)
    ADNI2.head()

In [47]:
# MERGING PROTEIN and DRUG
if switch == 3:

        # checking out the number of records for patients in ADNI2 phase, compared with other phases
        ADMCPATIENTDRUGCLASSES_20170512 = pd.read_csv('ADMCPATIENTDRUGCLASSES_20170512.csv')
        print(len(ADMCPATIENTDRUGCLASSES_20170512['Phase']))
        print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI2']))
        print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI3']))
        print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNIGO']))

        # select only ADNI2 data within MEDHIST
        drug_adni2 = ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI2']

        # renaming VISCODE2 column to VISCODE; this is okay, since we're only using ADNI2 data. 
        drug_adni2 = drug_adni2.rename(columns={'VISCODE2': 'VISCODE'})

        # create a list of all the drugs in ADNI2 
        drug_list_allviscode = [x for x in drug_adni2['NA']]
        # use a set to find all the unique drug names in the dataset
        all_drug_list_all_viscode = set(drug_list_allviscode)

        # create a list of blood-thinners from a google search
        blood_thinners_original = ['DABIGATRAN','PRADAXA','RIVAROXABAN','XARELTO','APIXABAN','ELIQUIS','HEPARIN',\
                                    'WARFARIN', 'COUMADIN', 'ASPIRIN']

        # a more extensive list of blood thinners found online
        blood_thinners = ['ACTRON DAMASON P LIQUPRIN PRESALIN ADVIL DARVON LODINE PROFEN AGGRENOX DASIN LORTAB RELAFEN \
        ALEVE DAYPRO MAGNAPRIN ROBAXISAL ALKA-SELTZER DHC PLUS MARNAL ROXIPRIN ANACIN DIAGESIC MEASURIN RUFEN ANAPROX\
        DICLOFENAC MECLOFENAMATE SALSALATE ANEXSIA DIPYRIDAMOLE MEFENAMIC SALETO ANODYNOS DISALCID MELOXICAM SALOCOL\
        ANSAID DOLABID MEPROBAMATE SOMA COMPOUND A.P.C. DOLPRIN MIDOL SPRIX NASAL SPRAY ARTROTEC DOXAPHENE MOBIC ST.\
        JOSEPH A.S.A. DRISTAN MOMENTUM SULINDAC ASCRIPTIN EASPRIN MOTRIN SUPAC ASPERGUM ECOTRIN NABUMETONE SYNALGOS \
        ASPIRIN EMAGRIN FORTE NAPRELAN TALWIN AXOTAL EMPIRIN NAPRAPAC TICAGRELOR BAC EQUAGESIC NAPROSYN TICLOPIDINE BAYER\
        EQUAZINE NAPROXEN TICLID BEXOPHENE ETODOLAC NORGESIC TOLECTIN BEXTRA EXCEDRIN NUPRIN TOLMETIN BRILINTA FELDENE\
        ORUDIS TORADOL BUFFERED ASPIRIN FENOPROFEN ORUVAIL TRENTAL BUFFAPRIN FIOGESIC OXAPROZIN TRIGESIC BUFFERIN FIORGEN\
        PF PABALATE TRILISATE BUFFINOL FIORINAL  ULTRAPRIN CAMA ARTHRITIS PERCODAN UNIPRO CATAFLAM GINKO BILOBA PERSANTINE\
        VANGUISH CELEBREX GEMNISYN PERSISTIN VICOPROFEN CHERACOL CAPSULES IBUPROFEN PIROXICAM VIMOVO CLINORIL INDOCIN PLAVIX\
        CLOPIDOGREL INDOMETHACIN PLETAL VOLTAREN COMBUNOX KETOPROFEN PONSTEL ZIPSOR CONGESPIRIN KETOROLAC PREVACID NAPRAPAC\
        ZORPIN COUMADIN WARFARIN PRADAXA DABIGATRAN ETEXILATE'.split(' ')]

        # combine the 2 blood thinner lists to create a main list of all blood thinners
        blood_thinners = blood_thinners[0]
        blood_thinners = blood_thinners + blood_thinners_original

        # create a list of the supplements of interest, including all possible name variations
        supplements_original = ['CALCIUM','CA', 'CA2+','VITAMIN D','VIT D','VITA D']

        # split the NA column to identify all listed medications
        drug_adni2['DRUGS'] = drug_adni2["NA"].apply(lambda x:str(x).split(","))


        for sup in ['VITAMIN D', 'CALCIUM', 'VIT D']:
            drug_adni2[sup] = drug_adni2.apply(lambda x: np.sum(sup in x['DRUGS']), axis=1)

        # function to identify whether blood thinners are in the medical history dataset entries
        def ANY_BT(row):
            ANY=0
            for bt in blood_thinners:
                if bt in row["DRUGS"]:
                    ANY=1
            return ANY

        # any_bt column to identify if patient is taking a blood thinner (yes = 1)
        drug_adni2["ANY BT"] = drug_adni2.apply(ANY_BT, axis=1)
        # column to identify if patient is taking vitamin D supplement (yes = 1)
        drug_adni2["VITAMIN D merge"] = drug_adni2.apply(lambda x: np.sum(x["VITAMIN D"]==1 or x["VIT D"]==1), axis=1)
        drug_adni2 = drug_adni2.drop(["VITAMIN D", "VIT D"], axis=1)
        drug_adni2
        print(drug_adni2.shape)

        cholesterol_drugs = 'ATORVASTATIN, LIPITOR, FLUVASTATIN, LESCOL, LESCOL XL, LOVASTATIN, MEVACOR, ALTOPREV, PRAVASTATIN, PRAVACHOL, ROSUVASTATIN, CRESTOR, SIMVASTATIN, ZOCOR, PITAVASTATIN, LIVALO'
        cholesterol_drugs = cholesterol_drugs.split(', ')

        def ANY_CHOL(row):
               ANY=0
               for ct in cholesterol_drugs:
                   if ct in row["DRUGS"]:
                       ANY=1
               return ANY
    
        drug_adni2['ANY CHOL'] = drug_adni2.apply(ANY_CHOL, axis=1)
        
        # define a new dataframe with medications, VISCODE, RID
        drug_adni2_three_col = drug_adni2[['CALCIUM','ANY BT','VITAMIN D merge', 'ANY CHOL','RID', "VISCODE"]]
        
        print(ADNI2.shape)
        ADNI2= pd.merge(ADNI2,drug_adni2_three_col, on=['RID', 'VISCODE'])
        ADNI2.head()

        
        ADNI_bio = pd.read_csv('UPENNBIOMK9_04_19_17.csv')
        ADNI2_bio = ADNI_bio[ADNI_bio['PHASE']=='ADNI2']
        ADNI2_bio_bl = ADNI2_bio[ADNI2_bio['VISCODE2']=='bl']
        ADNI2_bio_bl = ADNI2_bio_bl[['ABETA','TAU','PTAU', 'RID']]
        ADNI2_bio_bl.loc[ADNI2_bio_bl['ABETA']=='>1700', 'ABETA'] = 1700
        ADNI2_bio_bl.loc[ADNI2_bio_bl['PTAU']=='<8', 'PTAU'] = 8
        print("ADNI2 shape: ", ADNI2_bio_bl.shape)

        ADNI2 = ADNI2.merge(ADNI2_bio_bl, how='inner', left_on='RID', right_on='RID')
        print(ADNI2.shape)
        ADNI2.head()

14575
7417
0
967
(7417, 107)
(789, 94)
ADNI2 shape:  (702, 4)
(702, 101)


In [48]:
print(ADNI2.shape)

(702, 101)


In [49]:
# Replacement of -4,-1, and Unknown
import numpy as np
ADNI2 = ADNI2.replace('Unknown', np.nan)
ADNI2 = ADNI2.replace(-1, np.nan)
ADNI2 = ADNI2.replace(-4, np.nan)

In [50]:
if switch == 3:
    ADNI2.to_csv(path_or_buf='ADNI2_bl.csv', index=False)

# From now on DX


In [11]:
ADNI2_bl = ADNI2.copy()

In [12]:
ADNI2_bl.columns

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
       'DX_bl', 'AGE', 'PTGENDER',
       ...
       'Month', 'M', 'update_stamp', 'CALCIUM', 'ANY BT', 'VITAMIN D merge',
       'ANY CHOL', 'ABETA', 'TAU', 'PTAU'],
      dtype='object', length=101)

In [13]:
ADNI2_bl.shape

(702, 101)

In [14]:
ADNI2_bl= ADNI2_bl.drop(labels = ['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
                                  'Month_bl', 'Month', 'M','Years_bl','EXAMDATE_bl',
                                 'FDG', 'PIB', 'AV45', 'CDRSB',
                                 'MMSE', 'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
                                 'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
                                 'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
                                 'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
                                 'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'FLDSTRENG', 'FSVERSION',
                                 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal', 'Fusiform',
                                 'MidTemp', 'ICV', 'DX','update_stamp','ADAS11', 'ADAS13','FLDSTRENG_bl'],axis=1)
ADNI2_bl.columns

Index(['DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT',
       'PTMARRY', 'APOE4', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl',
       'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl',
       'RAVLT_perc_forgetting_bl', 'FAQ_bl', 'FSVERSION_bl', 'Ventricles_bl',
       'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl',
       'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl',
       'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'PIB_bl', 'AV45_bl',
       'CALCIUM', 'ANY BT', 'VITAMIN D merge', 'ANY CHOL', 'ABETA', 'TAU',
       'PTAU'],
      dtype='object')

In [15]:
# Calculate the % of each column that is NaNs and print in descending order
print((ADNI2_bl.isnull().sum()/len(ADNI2_bl['DX_bl'])).sort_values(ascending=False))

PIB_bl                      1.000000
MidTemp_bl                  0.136752
Fusiform_bl                 0.136752
Entorhinal_bl               0.136752
Hippocampus_bl              0.108262
Ventricles_bl               0.081197
WholeBrain_bl               0.048433
EcogSPOrgan_bl              0.044160
EcogPtOrgan_bl              0.032764
EcogSPDivatt_bl             0.029915
EcogSPVisspat_bl            0.021368
ICV_bl                      0.018519
FSVERSION_bl                0.018519
EcogSPPlan_bl               0.017094
RAVLT_forgetting_bl         0.015670
MOCA_bl                     0.012821
AV45_bl                     0.012821
EcogPtDivatt_bl             0.012821
RAVLT_learning_bl           0.012821
EcogPtVisspat_bl            0.009972
EcogSPTotal_bl              0.008547
ADAS13_bl                   0.005698
EcogPtPlan_bl               0.005698
PTETHCAT                    0.005698
EcogSPMem_bl                0.005698
EcogSPLang_bl               0.005698
EcogPtLang_bl               0.004274
E

In [16]:
# Drop columns with 40% or more of NaNs(AUTOMATED)
print(ADNI2_bl.shape)
ADNI2_bl = ADNI2_bl.dropna(thresh=0.6*len(ADNI2_bl), axis=1)
print(ADNI2_bl.shape)
print((ADNI2_bl.isnull().sum()/len(ADNI2_bl['DX_bl'])).sort_values(ascending=False))

(702, 50)
(702, 49)
Entorhinal_bl               0.136752
MidTemp_bl                  0.136752
Fusiform_bl                 0.136752
Hippocampus_bl              0.108262
Ventricles_bl               0.081197
WholeBrain_bl               0.048433
EcogSPOrgan_bl              0.044160
EcogPtOrgan_bl              0.032764
EcogSPDivatt_bl             0.029915
EcogSPVisspat_bl            0.021368
ICV_bl                      0.018519
FSVERSION_bl                0.018519
EcogSPPlan_bl               0.017094
RAVLT_forgetting_bl         0.015670
AV45_bl                     0.012821
RAVLT_learning_bl           0.012821
MOCA_bl                     0.012821
EcogPtDivatt_bl             0.012821
EcogPtVisspat_bl            0.009972
EcogSPTotal_bl              0.008547
EcogPtPlan_bl               0.005698
PTETHCAT                    0.005698
ADAS13_bl                   0.005698
EcogSPMem_bl                0.005698
EcogSPLang_bl               0.005698
EcogPtLang_bl               0.004274
EcogPtTotal_bl    

In [17]:
# ENCODE in an ORDER WAY
ADNI2_bl['DX_bl'] = ADNI2_bl['DX_bl'].replace('CN', 0)
ADNI2_bl['DX_bl']= ADNI2_bl['DX_bl'].replace('SMC', 0)
ADNI2_bl['DX_bl']= ADNI2_bl['DX_bl'].replace('EMCI', 1)
ADNI2_bl['DX_bl'] = ADNI2_bl['DX_bl'].replace('LMCI', 1)
ADNI2_bl['DX_bl'] = ADNI2_bl['DX_bl'].replace('AD', 2)
print(ADNI2_bl['DX_bl'].head(20))

0     0
1     0
2     0
3     0
4     0
5     2
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    2
19    2
Name: DX_bl, dtype: int64


In [18]:
ADNI2_bl['DX_bl'].value_counts()

1    315
0    255
2    132
Name: DX_bl, dtype: int64

In [19]:
ADNI2_bl.columns

Index(['DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT',
       'PTMARRY', 'APOE4', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl',
       'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl',
       'RAVLT_perc_forgetting_bl', 'FAQ_bl', 'FSVERSION_bl', 'Ventricles_bl',
       'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl',
       'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl',
       'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl', 'CALCIUM',
       'ANY BT', 'VITAMIN D merge', 'ANY CHOL', 'ABETA', 'TAU', 'PTAU'],
      dtype='object')

In [20]:
# Hot-one encoding for categorical
print(ADNI2_bl.shape)
if (switch == 0) or (switch == 2):
    categorical_predictors = ['PTETHCAT', 'PTRACCAT', 'PTMARRY','APOE4', 'PTGENDER', 'FSVERSION_bl']
    
if (switch == 1) or (switch == 3):
    categorical_predictors = ['PTETHCAT', 'PTRACCAT', 'PTMARRY','APOE4',
                              'PTGENDER', 'FSVERSION_bl',
                              'CALCIUM','ANY BT','VITAMIN D merge', 'ANY CHOL']                                                         

    
#Remove all categorical data with nan (only ~0.5%) 
missing_rows = np.nonzero(np.sum(ADNI2_bl[categorical_predictors].isnull(), axis=1))
ADNI2_bl = ADNI2_bl.drop(ADNI2_bl.index[missing_rows[0]])

#Hot-One Encoding of Categorical Predictors 
ADNI2_bl_expand = pd.get_dummies(ADNI2_bl, columns=categorical_predictors, drop_first=False)
ADNI2_bl_categorical = pd.get_dummies(ADNI2_bl[categorical_predictors] , columns=categorical_predictors, prefix = categorical_predictors , drop_first=False)
print(ADNI2_bl_expand.shape)
print(ADNI2_bl_categorical.shape)


(702, 49)
(684, 65)
(684, 26)


In [21]:
print(ADNI2_bl_expand.columns)

Index(['DX_bl', 'AGE', 'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl',
       'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl',
       'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'FAQ_bl',
       'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl',
       'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl', 'ABETA',
       'TAU', 'PTAU', 'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0', 'APOE4_1.0',
      

In [22]:
if  switch == 3:
    ADNI2_bl_expand.to_csv(path_or_buf='ADNI2_bl_expand.csv', index=False)

# ADNI2_bl_expand will be the input data for the three methods that deal with NaNs

### Removing All Missing Data

In [23]:
#R emoving all the rows with NaNs
ADNI2_bl_remove = ADNI2_bl_expand.copy()
print('Shape before delteing missing', ADNI2_bl_remove.shape)
ADNI2_bl_remove.dropna(axis=0, inplace=True)
print('Shape before delteing missing', ADNI2_bl_remove.shape) 
print('Data contains NaN?', ADNI2_bl_remove.isnull().any().any())

Shape before delteing missing (684, 65)
Shape before delteing missing (453, 65)
Data contains NaN? False


In [24]:
# Check that there no NaNs
ADNI2_bl_remove.isnull().sum()

DX_bl                                            0
AGE                                              0
PTEDUCAT                                         0
CDRSB_bl                                         0
ADAS11_bl                                        0
ADAS13_bl                                        0
MMSE_bl                                          0
RAVLT_immediate_bl                               0
RAVLT_learning_bl                                0
RAVLT_forgetting_bl                              0
RAVLT_perc_forgetting_bl                         0
FAQ_bl                                           0
Ventricles_bl                                    0
Hippocampus_bl                                   0
WholeBrain_bl                                    0
Entorhinal_bl                                    0
Fusiform_bl                                      0
MidTemp_bl                                       0
ICV_bl                                           0
MOCA_bl                        

In [25]:
# SAVING OF DATASETS (train and test)

ADNI2_bl_remove_train, ADNI2_bl_remove_test = split(ADNI2_bl_remove,0.75)

if switch == 0:
    ADNI2_bl_remove_train.to_csv(path_or_buf='ADNI2_bl_remove_train.csv', index=False)
    ADNI2_bl_remove_test.to_csv(path_or_buf='ADNI2_bl_remove_test.csv', index=False)
    
if switch == 1: 
    #put back the predictors
#     ADNI2_bl_remove_train = ADNI2_bl_remove_train[['CALCIUM','ANY BT','VITAMIN D merge','DX_bl']]
#     ADNI2_bl_remove_test = ADNI2_bl_remove_test[['CALCIUM','ANY BT','VITAMIN D merge','DX_bl']]
    ADNI2_bl_remove_train.to_csv(path_or_buf='ADNI2_bl_remove_train_DRUG.csv', index=False)
    ADNI2_bl_remove_test.to_csv(path_or_buf='ADNI2_bl_remove_test_DRUG.csv', index=False)
    

if switch == 2: 
    ADNI2_bl_remove_train.to_csv(path_or_buf='ADNI2_bl_remove_train_PROTEIN.csv', index=False)
    ADNI2_bl_remove_test.to_csv(path_or_buf='ADNI2_bl_remove_test_PROTEIN.csv', index=False)

if switch == 3: 
    ADNI2_bl_remove_train.to_csv(path_or_buf='ADNI2_bl_remove_train_DRUG+PROTEIN.csv', index=False)
    ADNI2_bl_remove_test.to_csv(path_or_buf='ADNI2_bl_remove_test_DRUG+PROTEIN.csv', index=False)



## Model-Based Imputation 
- Drop around 5% of Categorical predictors and use Linear Regression for missing continuous predictors

In [26]:
ADNI2_bl_model = ADNI2_bl_expand.copy()

In [27]:
ADNI2_bl_model.columns

Index(['DX_bl', 'AGE', 'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl',
       'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl',
       'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'FAQ_bl',
       'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl',
       'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl', 'ABETA',
       'TAU', 'PTAU', 'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0', 'APOE4_1.0',
      

In [28]:
#Manually select continuous predictors for differnet switchdx

if (switch == 0) or (switch == 1):
    continuous_predictors = ['AGE', 'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl',
                            'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl',
                            'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl',
                            'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl',
                            'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl',
                            'EcogPtPlan_bl', 'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl',
                            'EcogSPMem_bl', 'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl',
                            'EcogSPOrgan_bl', 'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl',
                            'AV45_bl']

if (switch == 2) or (switch == 3):
    continuous_predictors = ['ABETA','TAU','PTAU','AGE', 'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl',
                            'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl',
                            'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl',
                            'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl',
                            'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl',
                            'EcogPtPlan_bl', 'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl',
                            'EcogSPMem_bl', 'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl',
                            'EcogSPOrgan_bl', 'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl',
                            'AV45_bl']

In [29]:
# Create ADNI2_missing for imputation
ADNI2_bl_model_remove_categorical = ADNI2_bl_model[continuous_predictors]

# Create the Dataframe for imputation
ADNI2_bl_missing = pd.concat([ADNI2_bl_model_remove_categorical, ADNI2_bl_categorical], axis=1)


In [30]:
print(ADNI2_bl_model_remove_categorical.columns)
print(ADNI2_bl.columns)
print(ADNI2_bl_categorical.columns)

Index(['ABETA', 'TAU', 'PTAU', 'AGE', 'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl',
       'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl',
       'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'FAQ_bl',
       'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl',
       'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl'],
      dtype='object')
Index(['DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT',
       'PTMARRY', 'APOE4', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl',
       'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl',
       'RAVLT_perc_forgetting_bl', 'FAQ_bl', 'FSVERSION_bl', 'Ventricles_bl',
       'Hippocampus_bl', 'Whol

In [31]:
print(ADNI2_bl_model_remove_categorical.shape)
print(ADNI2_bl_categorical.shape)
print(ADNI2_bl.shape)

(684, 38)
(684, 26)
(684, 49)


In [32]:
full_dataframe =  copy(ADNI2_bl_missing)
missing_rows = []
for i in range(ADNI2_bl_missing.shape[0]):
    if pd.isnull(ADNI2_bl_missing.values[i]).any():
        missing_rows.append(i)
full_dataframe= full_dataframe.drop(full_dataframe.index[missing_rows], axis=0);

print('The origin number of data: {}'.format(len(ADNI2_bl_missing)))
print('The number of data that contains missing entries: {}'.format(len(missing_rows)))
print('The dimension of full dataframe: {}'.format(full_dataframe.shape))

# Check that full_dataframe has no missing values 
print(np.sum(np.sum(full_dataframe.isnull())))

The origin number of data: 684
The number of data that contains missing entries: 231
The dimension of full dataframe: (453, 64)
0


In [33]:
print(ADNI2_bl_missing.columns)

Index(['ABETA', 'TAU', 'PTAU', 'AGE', 'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl',
       'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl',
       'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'FAQ_bl',
       'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl',
       'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl',
       'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0', 'APOE4_1.0',
       'APOE4_2

In [34]:
# Model-Based Imputation - Linear Regression

lr = LinearRegression(fit_intercept = True)

for i in range(ADNI2_bl_missing.shape[0]):
    if ADNI2_bl_missing.iloc[i, :].isnull().any():
        missing_columns = ADNI2_bl_missing.columns[ADNI2_bl_missing.iloc[i, :].isnull()]
        y_imp = full_dataframe[missing_columns]
        X_imp = full_dataframe.drop(missing_columns, 1)
        lr.fit(X_imp,y_imp)
        X_missing = ADNI2_bl_missing.iloc[i, :].drop(missing_columns)
        y_hat = lr.predict(X_missing)
        ADNI2_bl_missing.loc[ADNI2_bl_missing.index[i], missing_columns] = y_hat[0] 
     

In [35]:
#Put back DX-bl and check that there are not missing

print(ADNI2_bl_expand.shape)
print(ADNI2_bl_missing.shape)
ADNI2_bl_missing['DX_bl'] = ADNI2_bl_expand['DX_bl']
print(ADNI2_bl_missing.shape)
ADNI2_bl_missing.isnull().sum()

(684, 65)
(684, 64)
(684, 65)


ABETA                                            0
TAU                                              0
PTAU                                             0
AGE                                              0
PTEDUCAT                                         0
CDRSB_bl                                         0
ADAS11_bl                                        0
ADAS13_bl                                        0
MMSE_bl                                          0
RAVLT_immediate_bl                               0
RAVLT_learning_bl                                0
RAVLT_forgetting_bl                              0
RAVLT_perc_forgetting_bl                         0
FAQ_bl                                           0
Ventricles_bl                                    0
Hippocampus_bl                                   0
WholeBrain_bl                                    0
Entorhinal_bl                                    0
Fusiform_bl                                      0
MidTemp_bl                     

In [36]:
# SAVING OF DATASETS (train and test)

ADNI2_bl_missing_train, ADNI2_bl_missing_test = split(ADNI2_bl_missing,0.75)

if switch == 0:
    ADNI2_bl_missing_train.to_csv(path_or_buf='ADNI2_bl_model_train.csv', index=False)
    ADNI2_bl_missing_test.to_csv(path_or_buf='ADNI2_bl_model_test.csv', index=False)
    
if switch == 1:
    ADNI2_bl_missing_train.to_csv(path_or_buf='ADNI2_bl_model_train_DRUG.csv', index=False)
    ADNI2_bl_missing_test.to_csv(path_or_buf='ADNI2_bl_model_test_DRUG.csv', index=False)
    
if switch == 2:
    ADNI2_bl_missing_train.to_csv(path_or_buf='ADNI2_bl_model_train_PROTEIN.csv', index=False)
    ADNI2_bl_missing_test.to_csv(path_or_buf='ADNI2_bl_model_test_PROTEIN.csv', index=False)

if switch == 3:
    ADNI2_bl_missing_train.to_csv(path_or_buf='ADNI2_bl_model_train_DRUG+PROTEIN.csv', index=False)
    ADNI2_bl_missing_test.to_csv(path_or_buf='ADNI2_bl_model_test_DRUG+PROTEIN.csv', index=False)

# Median Imputation¶

In [37]:
ADNI2_bl_mean = ADNI2_bl_expand.copy()
print(ADNI2_bl_mean.isnull().sum())

DX_bl                                             0
AGE                                               0
PTEDUCAT                                          0
CDRSB_bl                                          0
ADAS11_bl                                         2
ADAS13_bl                                         4
MMSE_bl                                           0
RAVLT_immediate_bl                                1
RAVLT_learning_bl                                 8
RAVLT_forgetting_bl                              11
RAVLT_perc_forgetting_bl                          1
FAQ_bl                                            2
Ventricles_bl                                    44
Hippocampus_bl                                   63
WholeBrain_bl                                    21
Entorhinal_bl                                    81
Fusiform_bl                                      81
MidTemp_bl                                       81
ICV_bl                                            0
MOCA_bl     

In [38]:
# Mean imputation with "Imputer" function
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
ADNI2_mean = imp.fit_transform(ADNI2_bl_mean)
header = ADNI2_bl_mean.columns
ADNI2_bl_mean_final = pd.DataFrame(data = ADNI2_mean, columns=header)
ADNI2_bl_mean_final.head()

,DX_bl,AGE,PTEDUCAT,CDRSB_bl,ADAS11_bl,ADAS13_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,...,PTGENDER_Male,FSVERSION_bl_Cross-Sectional FreeSurfer (5.1),CALCIUM_0,CALCIUM_1,ANY BT_0,ANY BT_1,VITAMIN D merge_0,VITAMIN D merge_1,ANY CHOL_0,ANY CHOL_1
0,0.0,66.9,17.0,0.0,10.0,15.0,29.0,42.0,5.0,2.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.0,67.5,16.0,0.0,3.0,5.0,30.0,42.0,7.0,2.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,0.0,68.5,20.0,0.0,4.0,7.0,30.0,61.0,6.0,3.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,80.2,19.0,0.0,3.0,10.0,29.0,36.0,8.0,7.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,0.0,71.8,15.0,0.5,4.0,6.0,28.0,44.0,8.0,4.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [39]:
#Check that there are no missing
print(ADNI2_bl_mean_final.isnull().sum())

DX_bl                                            0
AGE                                              0
PTEDUCAT                                         0
CDRSB_bl                                         0
ADAS11_bl                                        0
ADAS13_bl                                        0
MMSE_bl                                          0
RAVLT_immediate_bl                               0
RAVLT_learning_bl                                0
RAVLT_forgetting_bl                              0
RAVLT_perc_forgetting_bl                         0
FAQ_bl                                           0
Ventricles_bl                                    0
Hippocampus_bl                                   0
WholeBrain_bl                                    0
Entorhinal_bl                                    0
Fusiform_bl                                      0
MidTemp_bl                                       0
ICV_bl                                           0
MOCA_bl                        

In [40]:
# SAVING OF DATASETS (train and test)

ADNI2_bl_mean_train, ADNI2_bl_mean_test= split(ADNI2_bl_mean_final,0.75)

if switch == 0:
    ADNI2_bl_mean_train.to_csv(path_or_buf='ADNI2_bl_mean_train.csv', index=False)
    ADNI2_bl_mean_test.to_csv(path_or_buf='ADNI2_bl_mean_test.csv', index=False)
    
if switch == 1:
    ADNI2_bl_mean_train.to_csv(path_or_buf='ADNI2_bl_mean_train_DRUG.csv', index=False)
    ADNI2_bl_mean_test.to_csv(path_or_buf='ADNI2_bl_mean_test_DRUG.csv', index=False)
    
if switch == 2:
    ADNI2_bl_mean_train.to_csv(path_or_buf='ADNI2_bl_mean_train_PROTEIN.csv', index=False)
    ADNI2_bl_mean_test.to_csv(path_or_buf='ADNI2_bl_mean_test_PROTEIN.csv', index=False)

if switch == 3:
    ADNI2_bl_mean_train.to_csv(path_or_buf='ADNI2_bl_mean_train_DRUG+PROTEIN.csv', index=False)
    ADNI2_bl_mean_test.to_csv(path_or_buf='ADNI2_bl_mean_test_DRUG+PROTEIN.csv', index=False)
    